# Sentiment Analysis of Drug Reviews Using Natural Language Processing Techniques

## Problem Statement:

This study applies sentiment analysis and textual pattern extraction on drug reviews to identify not only user sentiment but also key factors such as side effects, ineffectiveness, or condition mismatch that contribute to negative medication experiences.

In [1]:
import pandas as pd
import re
import nltk
import string
import html
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

# download NLTK resources (do only once)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/afreenjunaidah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/afreenjunaidah/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/afreenjunaidah/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/afreenjunaidah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/afreenjunaidah/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/afreenjunaidah/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

## Basic Data Cleaning

1. Have basic understanding and cleaning of the data.

In [2]:
#df = pd.read_csv('drugsComTrain_raw.tsv' , sep='\t')
df = pd.read_csv('drugsComTrain_raw.tsv', sep='\t', on_bad_lines='skip')

df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [3]:
df.tail()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
161292,191035,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10.0,"May 31, 2015",125
161293,127085,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1.0,"November 1, 2011",34
161294,187382,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2.0,"March 15, 2014",35
161295,47128,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10.0,"September 19, 2015",79
161296,215220,Lubiprostone,"Constipation, Chronic","""I&#039;ve had chronic constipation all my adu...",9.0,"December 13, 2014",116


In [4]:
#Basic checks
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161297 entries, 0 to 161296
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   161297 non-null  int64  
 1   drugName     161297 non-null  object 
 2   condition    160398 non-null  object 
 3   review       161297 non-null  object 
 4   rating       161297 non-null  float64
 5   date         161297 non-null  object 
 6   usefulCount  161297 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 8.6+ MB


In [5]:
#Check for missing values
df.isna().sum()


Unnamed: 0       0
drugName         0
condition      899
review           0
rating           0
date             0
usefulCount      0
dtype: int64

In [6]:
#Remove missing values as it only contributes to ~0.9% of the data
df.dropna(subset=['condition'], inplace=True)
#Double check for missing values
df.isna().sum()

Unnamed: 0     0
drugName       0
condition      0
review         0
rating         0
date           0
usefulCount    0
dtype: int64

In [7]:
#Rename columns
df.rename(columns={'Unnamed: 0': 'patient_id'}, inplace=True)

In [8]:
#Check for duplicates
df.duplicated().sum()

0

In [9]:
#Change date to proper format
df['date'] = pd.to_datetime(df['date'], format='%B %d, %Y')


In [10]:
#Final check
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 160398 entries, 0 to 161296
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   patient_id   160398 non-null  int64         
 1   drugName     160398 non-null  object        
 2   condition    160398 non-null  object        
 3   review       160398 non-null  object        
 4   rating       160398 non-null  float64       
 5   date         160398 non-null  datetime64[ns]
 6   usefulCount  160398 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 9.8+ MB


In [11]:
#Create copy of cleaned data
df2=df.copy()

In [12]:
df2.head()

,patient_id,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,2012-05-20,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,2010-04-27,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,2009-12-14,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,2015-11-03,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,2016-11-27,37


## Text Preprocessing (from lab 3 & 4)

1. Sentence Segmentation

In [13]:
from nltk.tokenize import sent_tokenize

pd.set_option('display.max_colwidth', None)

df2['sentences'] = df2['review'].apply(sent_tokenize)
df2[['sentences']].head()

,sentences
0,"[""It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil""]"
1,"[""My son is halfway through his fourth week of Intuniv., We became concerned when he began this last week, when he started taking the highest dose he will be on., For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.), I called his doctor on Monday morning and she said to stick it out a few days., See how he did at school, and with getting up in the morning., The last two days have been problem free., He is MUCH more agreeable than ever., He is less emotional (a good thing), less cranky., He is remembering all the things he should., Overall his behavior is better., We have tried many different medications and so far this is the most effective.""]"
2,"[""I used to take another oral contraceptive, which had 21 pill cycle, and was very happy- very light periods, max 5 days, no other side effects., But it contained hormone gestodene, which is not available in US, so I switched to Lybrel, because the ingredients are similar., When my other pills ended, I started Lybrel immediately, on my first day of period, as the instructions said., And the period lasted for two weeks., When taking the second pack- same two weeks., And now, with third pack things got even worse- my third period lasted for two weeks and now it&#039;s the end of the third week- I still have daily brown discharge., The positive side is that I didn&#039;t have any other side effects., The idea of being period free was so tempting..., Alas.""]"
3,"[""This is my first time using any form of birth control., I&#039;m glad I went with the patch, I have been on it for 8 months., At first It decreased my libido but that subsided., The only downside is that it made my periods longer (5-6 days to be exact) I used to only have periods for 3-4 days max also made my cramps intense for the first two days of my period, I never had cramps before using birth control., Other than that in happy with the patch""]"
4,"[""Suboxone has completely turned my life around., I feel healthier, I&#039;m excelling at my job and I always have money in my pocket and my savings account., I had none of those before Suboxone and spent years abusing oxycontin., My paycheck was already spent by the time I got it and I started resorting to scheming and stealing to fund my addiction., All that is history., If you&#039;re ready to stop, there&#039;s a good chance that suboxone will put you on the path of great life again., I have found the side-effects to be minimal compared to oxycontin., I&#039;m actually sleeping better., Slight constipation is about it for me., It truly is amazing., The cost pales in comparison to what I spent on oxycontin.""]"


2. Tokenization

In [14]:
df2['tokens'] = df2['review'].apply(word_tokenize)
df2[['tokens']].head()

,tokens
0,"[``, It, has, no, side, effect, ,, I, take, it, in, combination, of, Bystolic, 5, Mg, and, Fish, Oil, '']"
1,"[``, My, son, is, halfway, through, his, fourth, week, of, Intuniv, ., We, became, concerned, when, he, began, this, last, week, ,, when, he, started, taking, the, highest, dose, he, will, be, on, ., For, two, days, ,, he, could, hardly, get, out, of, bed, ,, was, very, cranky, ,, and, slept, for, nearly, 8, hours, on, a, drive, home, from, school, vacation, (, very, unusual, for, him, ., ), I, called, his, doctor, on, Monday, morning, and, she, said, to, stick, it, out, a, few, days, ., See, how, he, did, at, school, ,, and, with, getting, up, in, ...]"
2,"[``, I, used, to, take, another, oral, contraceptive, ,, which, had, 21, pill, cycle, ,, and, was, very, happy-, very, light, periods, ,, max, 5, days, ,, no, other, side, effects, ., But, it, contained, hormone, gestodene, ,, which, is, not, available, in, US, ,, so, I, switched, to, Lybrel, ,, because, the, ingredients, are, similar, ., When, my, other, pills, ended, ,, I, started, Lybrel, immediately, ,, on, my, first, day, of, period, ,, as, the, instructions, said, ., And, the, period, lasted, for, two, weeks, ., When, taking, the, second, pack-, same, two, weeks, ., And, now, ,, ...]"
3,"[``, This, is, my, first, time, using, any, form, of, birth, control, ., I, &, #, 039, ;, m, glad, I, went, with, the, patch, ,, I, have, been, on, it, for, 8, months, ., At, first, It, decreased, my, libido, but, that, subsided, ., The, only, downside, is, that, it, made, my, periods, longer, (, 5-6, days, to, be, exact, ), I, used, to, only, have, periods, for, 3-4, days, max, also, made, my, cramps, intense, for, the, first, two, days, of, my, period, ,, I, never, had, cramps, before, using, birth, control, ., Other, than, that, in, happy, ...]"
4,"[``, Suboxone, has, completely, turned, my, life, around, ., I, feel, healthier, ,, I, &, #, 039, ;, m, excelling, at, my, job, and, I, always, have, money, in, my, pocket, and, my, savings, account, ., I, had, none, of, those, before, Suboxone, and, spent, years, abusing, oxycontin, ., My, paycheck, was, already, spent, by, the, time, I, got, it, and, I, started, resorting, to, scheming, and, stealing, to, fund, my, addiction, ., All, that, is, history, ., If, you, &, #, 039, ;, re, ready, to, stop, ,, there, &, #, 039, ;, s, a, good, chance, that, suboxone, ...]"


3. Case folding

In [15]:
#lowercasing
df2['tokens_lower'] = df2['tokens'].apply(lambda x: [word.lower() for word in x])
print(df2[['tokens_lower']].head())

#uppercasing
df2['tokens_upper'] = df2['tokens'].apply(lambda x: [word.upper() for word in x])
print(df2[['tokens_upper']].head())



                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           tokens_lower
0                                                                                                                                                                                                                                                                                                                                                                                                               

4. Punctuation Removal

In [16]:
df2['tokens_no_punct'] = df2['tokens_lower'].apply(lambda x: [word for word in x if word not in string.punctuation])
df2 [['tokens_no_punct']].head()

,tokens_no_punct
0,"[``, it, has, no, side, effect, i, take, it, in, combination, of, bystolic, 5, mg, and, fish, oil, '']"
1,"[``, my, son, is, halfway, through, his, fourth, week, of, intuniv, we, became, concerned, when, he, began, this, last, week, when, he, started, taking, the, highest, dose, he, will, be, on, for, two, days, he, could, hardly, get, out, of, bed, was, very, cranky, and, slept, for, nearly, 8, hours, on, a, drive, home, from, school, vacation, very, unusual, for, him, i, called, his, doctor, on, monday, morning, and, she, said, to, stick, it, out, a, few, days, see, how, he, did, at, school, and, with, getting, up, in, the, morning, the, last, two, days, have, been, problem, free, he, ...]"
2,"[``, i, used, to, take, another, oral, contraceptive, which, had, 21, pill, cycle, and, was, very, happy-, very, light, periods, max, 5, days, no, other, side, effects, but, it, contained, hormone, gestodene, which, is, not, available, in, us, so, i, switched, to, lybrel, because, the, ingredients, are, similar, when, my, other, pills, ended, i, started, lybrel, immediately, on, my, first, day, of, period, as, the, instructions, said, and, the, period, lasted, for, two, weeks, when, taking, the, second, pack-, same, two, weeks, and, now, with, third, pack, things, got, even, worse-, my, third, period, lasted, for, two, weeks, and, now, ...]"
3,"[``, this, is, my, first, time, using, any, form, of, birth, control, i, 039, m, glad, i, went, with, the, patch, i, have, been, on, it, for, 8, months, at, first, it, decreased, my, libido, but, that, subsided, the, only, downside, is, that, it, made, my, periods, longer, 5-6, days, to, be, exact, i, used, to, only, have, periods, for, 3-4, days, max, also, made, my, cramps, intense, for, the, first, two, days, of, my, period, i, never, had, cramps, before, using, birth, control, other, than, that, in, happy, with, the, patch, '']"
4,"[``, suboxone, has, completely, turned, my, life, around, i, feel, healthier, i, 039, m, excelling, at, my, job, and, i, always, have, money, in, my, pocket, and, my, savings, account, i, had, none, of, those, before, suboxone, and, spent, years, abusing, oxycontin, my, paycheck, was, already, spent, by, the, time, i, got, it, and, i, started, resorting, to, scheming, and, stealing, to, fund, my, addiction, all, that, is, history, if, you, 039, re, ready, to, stop, there, 039, s, a, good, chance, that, suboxone, will, put, you, on, the, path, of, great, life, again, i, have, found, the, side-effects, to, ...]"


5. Stopword Removal

In [17]:
stop_words = set(stopwords.words('english'))
df2['tokens_no_stop'] = df2['tokens'].apply(lambda x: [word for word in x if word not in stop_words])
df2[['tokens_no_stop']].head()

,tokens_no_stop
0,"[``, It, side, effect, ,, I, take, combination, Bystolic, 5, Mg, Fish, Oil, '']"
1,"[``, My, son, halfway, fourth, week, Intuniv, ., We, became, concerned, began, last, week, ,, started, taking, highest, dose, ., For, two, days, ,, could, hardly, get, bed, ,, cranky, ,, slept, nearly, 8, hours, drive, home, school, vacation, (, unusual, ., ), I, called, doctor, Monday, morning, said, stick, days, ., See, school, ,, getting, morning, ., The, last, two, days, problem, free, ., He, MUCH, agreeable, ever, ., He, less, emotional, (, good, thing, ), ,, less, cranky, ., He, remembering, things, ., Overall, behavior, better, ., We, tried, many, different, medications, far, effective, ., '']"
2,"[``, I, used, take, another, oral, contraceptive, ,, 21, pill, cycle, ,, happy-, light, periods, ,, max, 5, days, ,, side, effects, ., But, contained, hormone, gestodene, ,, available, US, ,, I, switched, Lybrel, ,, ingredients, similar, ., When, pills, ended, ,, I, started, Lybrel, immediately, ,, first, day, period, ,, instructions, said, ., And, period, lasted, two, weeks, ., When, taking, second, pack-, two, weeks, ., And, ,, third, pack, things, got, even, worse-, third, period, lasted, two, weeks, &, #, 039, ;, end, third, week-, I, still, daily, brown, discharge, ., The, positive, side, I, &, #, 039, ...]"
3,"[``, This, first, time, using, form, birth, control, ., I, &, #, 039, ;, glad, I, went, patch, ,, I, 8, months, ., At, first, It, decreased, libido, subsided, ., The, downside, made, periods, longer, (, 5-6, days, exact, ), I, used, periods, 3-4, days, max, also, made, cramps, intense, first, two, days, period, ,, I, never, cramps, using, birth, control, ., Other, happy, patch, '']"
4,"[``, Suboxone, completely, turned, life, around, ., I, feel, healthier, ,, I, &, #, 039, ;, excelling, job, I, always, money, pocket, savings, account, ., I, none, Suboxone, spent, years, abusing, oxycontin, ., My, paycheck, already, spent, time, I, got, I, started, resorting, scheming, stealing, fund, addiction, ., All, history, ., If, &, #, 039, ;, ready, stop, ,, &, #, 039, ;, good, chance, suboxone, put, path, great, life, ., I, found, side-effects, minimal, compared, oxycontin, ., I, &, #, 039, ;, actually, sleeping, better, ., Slight, constipation, ., It, truly, amazing, ., The, cost, pales, comparison, I, spent, ...]"


7. Stemming

In [19]:
stemmer = PorterStemmer()
df2['stemmed_tokens'] = df2['tokens_no_abbrev'].apply(lambda x: [stemmer.stem(word) for word in x])
df2[['stemmed_tokens']].head()


,stemmed_tokens
0,"[``, it, side, effect, ,, i, take, combin, bystol, 5, mg, fish, oil, '']"
1,"[``, my, son, halfway, fourth, week, intuniv, ., we, becam, concern, began, last, week, ,, start, take, highest, dose, ., for, two, day, ,, could, hardli, get, bed, ,, cranki, ,, slept, nearli, 8, hour, drive, home, school, vacat, (, unusu, ., ), i, call, doctor, monday, morn, said, stick, day, ., see, school, ,, get, morn, ., the, last, two, day, problem, free, ., he, much, agreeabl, ever, ., he, less, emot, (, good, thing, ), ,, less, cranki, ., he, rememb, thing, ., overal, behavior, better, ., we, tri, mani, differ, medic, far, effect, ., '']"
2,"[``, i, use, take, anoth, oral, contracept, ,, 21, pill, cycl, ,, happy-, light, period, ,, max, 5, day, ,, side, effect, ., but, contain, hormon, gestoden, ,, avail, us, ,, i, switch, lybrel, ,, ingredi, similar, ., when, pill, end, ,, i, start, lybrel, immedi, ,, first, day, period, ,, instruct, said, ., and, period, last, two, week, ., when, take, second, pack-, two, week, ., and, ,, third, pack, thing, got, even, worse-, third, period, last, two, week, &, #, 039, ;, end, third, week-, i, still, daili, brown, discharg, ., the, posit, side, i, &, #, 039, ...]"
3,"[``, thi, first, time, use, form, birth, control, ., i, &, #, 039, ;, glad, i, went, patch, ,, i, 8, month, ., at, first, it, decreas, libido, subsid, ., the, downsid, made, period, longer, (, 5-6, day, exact, ), i, use, period, 3-4, day, max, also, made, cramp, intens, first, two, day, period, ,, i, never, cramp, use, birth, control, ., other, happi, patch, '']"
4,"[``, suboxon, complet, turn, life, around, ., i, feel, healthier, ,, i, &, #, 039, ;, excel, job, i, alway, money, pocket, save, account, ., i, none, suboxon, spent, year, abus, oxycontin, ., my, paycheck, alreadi, spent, time, i, got, i, start, resort, scheme, steal, fund, addict, ., all, histori, ., if, &, #, 039, ;, readi, stop, ,, &, #, 039, ;, good, chanc, suboxon, put, path, great, life, ., i, found, side-effect, minim, compar, oxycontin, ., i, &, #, 039, ;, actual, sleep, better, ., slight, constip, ., it, truli, amaz, ., the, cost, pale, comparison, i, spent, ...]"


9. Word Sense Disambiguation

In [21]:
from nltk.wsd import lesk

df2['wsd'] = df2['review'].apply(lambda x: lesk(word_tokenize(x), 'depression', 'n'))
df2['wsd_def'] = df2['wsd'].apply(lambda x: x.definition() if x else "No definition found")

df2[['wsd', 'wsd_def']].head()


,wsd,wsd_def
0,Synset('depressive_disorder.n.01'),a state of depression and anhedonia so severe as to require clinical intervention
1,Synset('depressive_disorder.n.01'),a state of depression and anhedonia so severe as to require clinical intervention
2,Synset('depressive_disorder.n.01'),a state of depression and anhedonia so severe as to require clinical intervention
3,Synset('depressive_disorder.n.01'),a state of depression and anhedonia so severe as to require clinical intervention
4,Synset('depression.n.05'),a period during the 1930s when there was a worldwide economic depression and mass unemployment


12. Summarization

In [22]:
from collections import Counter

def generate_summary(text, num_sentences=3):
    # Break text into sentences
    sentence_list = sent_tokenize(text)
    if len(sentence_list) <= num_sentences:
        return text  # not enough to summarize

    #Count word frequencies (excluding stopwords and punctuation)
    stop_words = set(stopwords.words('english'))
    word_scores = Counter()

    for sentence in sentence_list:
        for word in word_tokenize(sentence.lower()):
            if word.isalpha() and word not in stop_words:
                word_scores[word] += 1

    #Score each sentence by summing word scores
    ranked_sentences = []
    for sentence in sentence_list:
        score = sum(word_scores.get(word.lower(), 0) for word in word_tokenize(sentence) if word.isalpha())
        ranked_sentences.append((sentence, score))

    #Sort sentences by score and return the top ones
    ranked_sentences.sort(key=lambda x: x[1], reverse=True)
    top_sentences = [sent for sent, _ in ranked_sentences[:num_sentences]]
    return ' '.join(top_sentences)

In [23]:
sample = df2['review'].iloc[1]
print("Original:\n", sample)
print("\nSummary:\n", generate_summary(sample, 2))

Original:
 "My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. 
We have tried many different medications and so far this is the most effective."

Summary:
 For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) We became concerned when he began this last week, when he started taking the highest